# Loading in csv files and creating the initial dataframe

In [70]:
import pandas as pd

# Reading in CSVs for each country
usa_df = pd.read_csv("regional-us-weekly-2025-07-17.csv")
usa_df["country"] = "USA"

canada_df = pd.read_csv("regional-ca-weekly-2025-07-17.csv")
canada_df["country"] = "Canada"

nigeria_df = pd.read_csv("regional-ng-weekly-2025-07-17.csv")
nigeria_df["country"] = "Nigeria"

sa_df = pd.read_csv("regional-za-weekly-2025-07-17.csv")
sa_df["country"] = "South Africa"

uk_df = pd.read_csv("regional-gb-weekly-2025-07-17.csv")
uk_df["country"] = "UK"

france_df = pd.read_csv("regional-fr-weekly-2025-07-17.csv")
france_df["country"] = "France"

brazil_df = pd.read_csv("regional-br-weekly-2025-07-17.csv")
brazil_df["country"] = "Brazil"

colombia_df = pd.read_csv("regional-co-weekly-2025-07-17.csv")
colombia_df["country"] = "Colombia"

india_df = pd.read_csv("regional-in-weekly-2025-07-17.csv")
india_df["country"] = "India"

sk_df = pd.read_csv("regional-kr-weekly-2025-07-17.csv")
sk_df["country"] = "South Korea"

# concat for one dataframe
final_df = pd.concat([usa_df, canada_df, nigeria_df, sa_df, uk_df, france_df, brazil_df, colombia_df, india_df, sk_df])


In [71]:
final_df

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,country
0,1,spotify:track:5BZsQlgw21vDOAjoqkNgKb,Justin Bieber,DAISIES,ILH Production Co. LLC / Def Jam Recordings,1,-1,1,14380416,USA
1,2,spotify:track:1CPZ5BxNNd0n0nF4Orb9JS,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo...",Golden,K-Pop Demon Hunters,1,1,4,11130174,USA
2,3,spotify:track:1I37Zz2g3hk9eWxaNkj031,"Saja Boys, Andrew Choi, Neckwav, Danny Chung, ...",Your Idol,K-Pop Demon Hunters,2,2,4,10110068,USA
3,4,spotify:track:29iva9idM6rFCPUlu7Rhxl,Justin Bieber,YUKON,ILH Production Co. LLC / Def Jam Recordings,4,-1,1,8229734,USA
4,5,spotify:track:3RXUgPNIbUgFxsDWuBQEt6,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo...",How It’s Done,K-Pop Demon Hunters,5,6,4,7974883,USA
...,...,...,...,...,...,...,...,...,...,...
195,196,spotify:track:2kIUILBPlz4exX9xIFS275,ILLIT,Lucky Girl Syndrome,BELIFT LAB,23,180,69,55147,South Korea
196,197,spotify:track:2lsRp40mCRkhqULWdMbOzn,Lim Young Woong,I Love You,물고기뮤직,15,150,166,54900,South Korea
197,198,spotify:track:4iJprGt1rt5iy0sxXXaRWn,"D-Hack, PATEKO",OHAYO MY NIGHT,NEOKIDZ,16,200,70,54800,South Korea
198,199,spotify:track:3KcHHPN23AwgnSkq7TouwW,"aespa, Flo Milli",Dirty Work (feat. Flo Milli),aespa,45,82,3,54595,South Korea


In [52]:
print(final_df.columns)

Index(['rank', 'uri', 'artist_names', 'track_name', 'source', 'peak_rank',
       'previous_rank', 'weeks_on_chart', 'streams', 'country'],
      dtype='object')


In [73]:
# Extract the track ID from the Spotify URI using regex
final_df['track_id'] = final_df['uri'].str.extract(r'spotify:track:([a-zA-Z0-9]+)')

# Accessing spotify API to get more in depth data

In [54]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [74]:
from dotenv import load_dotenv
import os
import time

# Load the environment variables from .env file
load_dotenv()

# Access the values
client_id = os.getenv("SPOTIPY_CLIENT_ID")
client_secret = os.getenv("SPOTIPY_CLIENT_SECRET")

# Initialize Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret))


In [75]:
# Define a function that gets artist info for a given track ID
def get_artist_info(track_id):
    try:
        # Get track details from the Spotify API
        track = sp.track(track_id)
        
        # Get the first artist's ID
        artist_id = track['artists'][0]['id']
        
        # Use that ID to fetch the artist's full info
        artist_data = sp.artist(artist_id)
        
        # Return artist name, genres, and popularity
        return pd.Series([
            artist_data['name'],
            artist_data['genres'],
            artist_data['popularity']
        ])
    except Exception as e:
        print(f"Error for {track_id}: {e}")
        return pd.Series([None, None, None])

In [76]:
# Add delay to avoid hitting API rate limits (max 10 requests/sec for client credentials flow)
def get_artist_info_safe(track_id):
    time.sleep(0.1)  # Adjust as needed
    return get_artist_info(track_id)

# Apply to the entire final_df
final_df[['main_artist', 'artist_genres', 'artist_popularity']] = final_df['track_id'].apply(get_artist_info_safe)

Error for 6393yGahSJ9slVdOwSxOVR: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)
Error for 2CeKVsFFXG4QzA415QygGb: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)


In [77]:
final_df.to_csv("spotify_enriched.csv", index=False)